In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
from plaid.compression.hourglass_vq import HourglassVQLightningModule

In [4]:
model = HourglassVQLightningModule.load_from_checkpoint(
    "/homefs/home/lux70/storage/plaid/checkpoints/hourglass_vq/x1zoydo3/last.ckpt"
)

using quantizer tanh


In [ ]:
device = torch.device("c

In [ ]:
from plaid.